In [1]:
import datetime as dt
import wget
import os
import pandas as pd
import numpy as np
from zipfile import ZipFile

In [2]:
# só extrai o arquivo dado como parâmetro para a pasta ./Dados/temp
def extrai_arquivo(arq):
    try:
        ZipFile(arq, 'r').extractall('./Dados/temp/')
    except:
        print(f"erro ao extrair {arq}")

In [3]:
# baixa dados, concatena em um dataframe só e salva em um .csv
# limpa depois
def baixa_e_concatena(ticker, timeframe, ano_inicial):
    ano_corrente, mes_corrente, dia_corrente = [dt.date.today().year, dt.date.today().month, dt.date.today().day]
    
    # baixa dados da binance conforme ticker e timeframe selecionados para a pasta ./Dados/
    # timeframes disponiveis: 12h 15m 1d 1h 1m 1mo 1s 1w 2h 30m 3d 3m 4h 5m 6h 8h
    # tickers disponiveis: https://data.binance.vision/?prefix=data/spot/monthly/klines/
    url = "https://data.binance.vision/data/spot/monthly/klines/"
    if not os.path.exists(f"./Dados/Processados/{ticker}-{timeframe}.csv"):
        for ano in range(ano_inicial, ano_corrente+1):
            for mes in range(1,12+1):
                mes = str(mes).zfill(2)
                if not ((os.path.exists(f"./Dados/temp/{ticker}-{timeframe}-{ano}-{mes}.zip"))):
                    try:
                        wget.download(f"{url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip"
                                      , out = f"./Dados/temp/")
                        pass
                    except:
                        print(f"\nFalha ao baixar {url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip")
                else:
                    print(f"{ano}/{mes} já baixado")
    else:
        print(f"{ticker}-{timeframe} já processado")
        return
    
    # cria uma lista de arquivos do ticker e timeframe selecionado
    lista_arquivos = os.listdir("./Dados/temp/")
    lista_arquivos = [x for x in lista_arquivos if x.startswith(f"{ticker}-{timeframe}")]
    lista_arquivos[-5:]
    
    # cria um dataframe vazio pra colocar todos os dados dentro
    nomes = ["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume"
                                 ,"Number of trades","Taker buy base asset volume","Taker buy quote asset volume","Ignore"]
    df = pd.DataFrame(columns = nomes)
    
    # concatena tudo em um CSV e deixa na pasta ./Dados/Processados/
    for arq in lista_arquivos:
        extrai_arquivo(f"./Dados/temp/{arq}")
        df = pd.concat([df, pd.read_csv(f'./Dados/temp/{arq[:-4]}.csv', sep=',',decimal='.'
                                   , encoding='latin1', names=nomes, header=None)], ignore_index=True, copy=False)
        os.remove(f"./Dados/temp/{arq[:-4]}.csv")
    df.drop("Ignore", inplace=True, axis=1)
    df.set_index("Open time", inplace=True)
    df.to_csv(f"./Dados/Processados/{ticker}-{timeframe}.csv")
    
    print(f"./Dados/Processados/{ticker}-{timeframe}.csv")
    
    # deleta tudo que é temporario e já foi processado
    for arq in lista_arquivos:
        os.remove(f"./Dados/temp/{arq}")
    
    return

In [4]:
# o de 1s demora muito. Pesa 16GB o arquivo final enquanto o de 1m pesa 300MB. tem que ver se vale a pena mesmo.
ticker = "BTCUSDT"
timeframes = ["1m", "5m", "15m", "30m", "1h", "2h", "4h", "8h", "1d"] # removi o "1s" pq o csv somente já pesa 16GB. inviável
for timeframe in timeframes:
    baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017)

BTCUSDT-1m já processado
BTCUSDT-5m já processado
BTCUSDT-15m já processado
BTCUSDT-30m já processado
BTCUSDT-1h já processado
BTCUSDT-2h já processado
BTCUSDT-4h já processado
BTCUSDT-8h já processado
BTCUSDT-1d já processado


In [5]:
# cria um dicionário de dataframes. Pra acessar é só falar o timeframe desejado
# transforma em numpy no final
dfs = {}
for timeframe in timeframes:
    dfs[timeframe] = (pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1]).to_numpy()
dfs["1m"]

array([[1.71636400e+04, 1.66985280e+12],
       [1.71648700e+04, 1.66985274e+12],
       [1.71612600e+04, 1.66985268e+12],
       ...,
       [4.28056000e+03, 1.50294258e+12],
       [4.26148000e+03, 1.50294252e+12],
       [4.26148000e+03, 1.50294246e+12]])

In [8]:
def cria_janela(linha, dfs=dfs, timeframes=timeframes):
    # cria janela vazia. precisa ter 1 coluna a mais que a qtd de timeframes
    df_janela = np.full(shape= (lookback, len(timeframes)+1), fill_value=np.nan)
    # coloca o preco de fechamento de 1m na primeira coluna e o close time de 1m na ultima coluna
    df_janela[:, [0,-1]] = dfs[timeframes[0]][linha:linha+lookback]

    i = 1
    for timeframe in timeframes[1:]:
        # pega o numero da linha o primeiro closetime do timeframe é igual ou menor que o da janela atual
        index = (dfs[timeframe][:,1] <= df_janela[0,-1]).argmax()
        # coloca as 20 (lookback) linhas subsequentes dentro da janela
        df_janela[:, i] = dfs[timeframe][index:index+lookback, 0]
        i += 1

    # deleta coluna de close time e retorna
    df_janela = np.delete(df_janela, obj=-1, axis=1)
    return(df_janela)

In [10]:
%%time
# numpy
# resultado bate
# tentar meter um paralelismo. demora 5 min pra rodar atualmente

linha = 0
lookback = 20

print(len(dfs[timeframes[0]])-lookback)
for linha in range(10000):
#for linha in range(len(dfs[timeframes[0]])-lookback):
    df_janela = cria_janela(linha, dfs, timeframes)
    
    # define se é long ou short
    # create_gaf(df_janela)
    # salva na pasta long ou short

df_janela

2773339
Wall time: 1.27 s


array([[16563.28, 16573.84, 16544.65, 16564.07, 16564.07, 16603.11,
        16603.11, 16603.11, 16603.11],
       [16573.84, 16544.65, 16564.07, 16565.8 , 16603.11, 16488.41,
        16473.32, 16427.11, 16226.94],
       [16573.3 , 16557.74, 16555.82, 16603.11, 16512.51, 16473.32,
        16427.11, 16583.84, 15781.29],
       [16566.72, 16593.01, 16565.8 , 16572.52, 16488.41, 16391.62,
        16585.77, 16226.94, 16280.23],
       [16565.73, 16564.07, 16620.97, 16512.51, 16617.47, 16427.11,
        16583.84, 16241.3 , 16700.68],
       [16558.25, 16542.21, 16603.11, 16469.5 , 16473.32, 16394.33,
        16586.55, 15760.42, 16700.45],
       [16544.65, 16547.48, 16615.26, 16488.41, 16527.81, 16585.77,
        16226.94, 15781.29, 16692.56],
       [16546.79, 16555.82, 16572.52, 16540.5 , 16391.62, 16566.38,
        16168.38, 16130.45, 16662.76],
       [16553.25, 16553.32, 16525.82, 16617.47, 16334.35, 16583.84,
        16241.3 , 16039.73, 16900.57],
       [16544.83, 16564.99, 16512.51,

# Problemas:
- Se for usar GAF image vai ter uma imagem 20x20 por coluna (supondo que estamos olhando 20 períodos no passado por vez)
- Não está claro se 1D CNN é uma boa alternativa para timeseries.

# Soluções:
#### 1
- Usar GAF de qualquer maneira para tentar replicar os resultados do trabalho do BARRA, usando apenas os preços de fechamento de 4 timeframes e ver se os resultados melhoram com CSVM
- Depois se eu quiser adiciono mais dados, mas a imagem vai ficar bem grande. (5 colunas de t-20 resulta em uma imagem de 100x100). GAF usando mais dados seria o melhor dos mundos.

#### 2
- Usar 1D CNN com todos os dados disponíveis
- Metodo completamente diferente do do BARRA. Não usa GAF images. Provavelmente mais fácil

### A partir daqui são só testes

In [ ]:
# gera pra todos os tempos
# define se é long ou short
# transforma em gaf
# salva como imagem nas pastas long e short
# passa pro keras imagedatagenerator

## OLD

In [50]:
# cria um dicionário de dataframes. Pra acessar é só falar o timeframe desejado
dfs = {}
for timeframe in timeframes:
    dfs[timeframe] = (pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1])
dfs["1m"]

,Close,Close time
2773358,17163.64,1669852799999
2773357,17164.87,1669852739999
2773356,17161.26,1669852679999
2773355,17167.16,1669852619999
2773354,17173.33,1669852559999
...,...,...
4,4261.48,1502942699999
3,4261.48,1502942639999
2,4280.56,1502942579999
1,4261.48,1502942519999


In [46]:
%%time

# versao com query melhorada
# confirmei que funciona batendo o resultado com o da versao antiga e lenta
# ver se dá pra usar só numpy (fica mto escroto)
# e depois paralelismo

lookback = 20
linha = 0

colunas = [f"Close_{timeframe}" for timeframe in timeframes]
colunas.append("Close time")

# pra todas as linhas
print("Num linhas: ", len(dfs[timeframes[0]])-lookback)
for linha in range(100):
#for linha in range(len(df)-lookback):
    # cria o esqueleto da janela com o primeiro timeframe
    df_janela = pd.DataFrame(np.nan, index=range(0,lookback), columns= colunas)
    df_janela[[colunas[0], colunas[-1]]] = dfs[timeframes[0]].iloc[linha:linha+lookback].reset_index(drop=True)
    # junta o preço de fechamento pra todos os timeframes
    i = 1
    for timeframe in timeframes[1:]:
        # pega o numero da linha o primeiro closetime do timeframe é igual ou menor que o da janela atual
        index = (dfs[timeframe].iloc[:,1] <= df_janela.iat[0,-1]).argmax()
        # coloca as 20 (lookback) linhas subsequentes dentro da janela
        df_janela.iloc[:, i] = dfs[timeframe].iloc[index:index+lookback, 0].reset_index(drop=True)
        i += 1
    # remove a coluna close time, que foi utilizada pra sincronizar cada timeframe
    df_janela.drop("Close time", inplace=True, axis=1)
    # define se é long ou short
    # create_gaf(df_janela)
    # salva na pasta long ou short


df_janela

Num linhas:  2773339
Wall time: 276 ms


,Close_1m,Close_5m,Close_15m,Close_30m,Close_1h,Close_2h,Close_4h,Close_8h,Close_1d
0,17089.85,17084.43,17095.53,17106.65,17106.65,17106.65,17062.85,16865.64,16442.53
1,17084.43,17095.53,17106.65,17041.71,17097.19,17062.85,16865.64,16884.18,16212.91
2,17083.10,17104.30,17064.97,17097.19,17062.85,16789.66,16879.32,16442.53,16428.78
3,17086.97,17101.14,17041.71,17064.40,16921.44,16865.64,16884.18,16393.48,16458.57
4,17083.55,17106.65,17050.64,17062.85,16789.66,16810.12,16845.25,16463.31,16522.14
5,17086.61,17085.95,17097.19,17115.80,16861.98,16879.32,16442.53,16212.91,16598.95
6,17095.53,17075.33,17043.54,16921.44,16865.64,16879.49,16426.56,16146.26,16603.11
7,17098.22,17064.97,17064.40,16742.05,16847.01,16884.18,16393.48,16222.06,16226.94
8,17092.02,17058.20,17044.23,16789.66,16810.12,16864.10,16497.64,16428.78,15781.29
9,17094.80,17057.79,17062.85,16765.70,16777.24,16845.25,16463.31,16574.28,16280.23


In [85]:
# criação de DF gigante e cheio de nan's

def une_timeframes(timeframes=timeframes):
    # junta todos os timeframes em um df só

    # pega o primeiro timeframe com as colunas desejadas
    df = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframes[0]}.csv", index_col = "Close time"
                     , usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
    #df.index = pd.to_datetime(df.index, unit="ms")

    # coloca o sufixo nas colunas do menor timeframe
    df.columns = df.columns+f"_{timeframes[0]}"
    print(timeframes[0])

    # junta todos os outros timeframes e coloca o sufixo correto
    for timeframe in timeframes[1:]:
        print(timeframe)
        df2 = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", index_col = "Close time",
                         usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
        df2.columns = df2.columns+f"_{timeframe}"
        df = df.merge(df2, how="left", on=["Close time"])#, suffixes=(None, f"_{timeframe}"))


    # transforma o index em datetime
    df.index = pd.to_datetime(df.index, unit="ms")

    #inverte a ordem do df pq vamos usar ele do fim pro começo pra fazer as janelas
    df.sort_index(axis=0, inplace = True, ascending=False)
    return(df)
df = une_timeframes(timeframes)
df

1m
5m
15m
30m
1h
2h
4h
8h
1d


,Open_1m,High_1m,Low_1m,Close_1m,Volume_1m,Number of trades_1m,Open_5m,High_5m,Low_5m,Close_5m,...,Low_8h,Close_8h,Volume_8h,Number of trades_8h,Open_1d,High_1d,Low_1d,Close_1d,Volume_1d,Number of trades_1d
Close time,,,,,,,,,,,,,,,,,,,,,
2022-11-30 23:59:59.999,17164.11,17167.99,17160.00,17163.64,143.984230,3565,17175.50,17215.59,17150.70,17163.64,...,16705.0,17163.64,115168.64732,2458897.0,16442.91,17249.0,16428.3,17163.64,303019.80719,6519330.0
2022-11-30 23:58:59.999,17161.26,17170.27,17159.51,17164.87,129.203750,3760,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-30 23:57:59.999,17165.50,17170.80,17156.77,17161.26,160.506230,4056,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-30 23:56:59.999,17173.33,17175.40,17150.70,17167.16,322.428780,7797,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-30 23:55:59.999,17175.50,17215.59,17170.01,17173.33,531.365170,6445,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-17 04:04:59.999,4261.48,4261.48,4261.48,4261.48,0.140796,1,4261.48,4280.56,4261.48,4261.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 04:03:59.999,4261.48,4261.48,4261.48,4261.48,0.012008,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 04:02:59.999,4280.56,4280.56,4280.56,4280.56,0.261074,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
%%time
# solução de janela usando uma df gigante. Bem mais lento
# cria janela de precos
lookback = 20
linha = 0
# tentar fazer um loop pra pegar tudo

# pra todas as linhas
print("Num linhas: ", len(df)-lookback)
for linha in [99]:
#for linha in range(len(df)-lookback):
    # zera df_janela
    df_janela = pd.DataFrame()
    for timeframe in timeframes:
        # insere na ultima coluna; com o nome de Close_timeframe ;
        # df a partir da linha atual até linha atual + lookback
        #df_janela[f"Close_{timeframe}"] = df[f"Close_{timeframe}"].iloc[linha:].dropna().iloc[:lookback].reset_index(drop=True)
        df_janela.insert(len(df_janela.columns),(f"Close_{timeframe}"),df[f"Close_{timeframe}"].iloc[linha:].dropna().iloc[:lookback].to_list())
    #print(linha)
    # define se é long ou short
    # create_gaf(df_janela)
    # salva na pasta long ou short
df_janela
df_janela2 = df_janela
# funciona, mas está muito lento
# se tirar o dropna vai de 4,4 seg pra 190 ms
# não unir mais tudo

Num linhas:  2773339
Wall time: 43.4 ms


In [ ]:
import pyts
def create_gaf(ts):
    """
    :param ts:
    :return:
    """
    data = dict()
    gadf = GramianAngularField(method='difference', image_size=ts.shape[0])
    data['gadf'] = gadf.fit_transform(pd.DataFrame(ts).T)[0] # ts.T)
    return data